# Deep Learning Book

# Chapter 10 Sequence Modeling

## Teacher Forcing, p372

A RNN whose only recurrence is the feedback connection from the output of the hidden layer, not the hidde layer itself, is **less powerful** than those which have direct connection from hidden layer $h^{(t-1)}$ to $h^{(t)}$. p370.

The advantage of eliminating hidden-to-hidden recurrence is that, for any loss function based on comparing the prediction at time $t$ to the training target at time $t$, all the time steps are decoupled. Tranining can thus be parallelized, with gradient for each step t computed in isolation.

Teacher forcing is a procedure that emerges from the maximum likelihood criterion, in which during training the model receives the ground truth output $y^{(t)}$ as input at time $t+1$, instead of the output from the previous step, thus parallelizing training.

## BPTT

The backprop algorithm applied to the unrolled graph with $\mathbb{O}(\tau)$ cost is called **back-propagation through time (BPTT)**. As soon as the hidden units become a function of earlier time steps, the BPTT algorithm is necessary.

Example is based on the RNN architecture given in figure 10.3 on page 369. This network is described as:

$$
\begin{aligned}
a^{(t)} &= b + W h^{(t-1)} + U x^{(t)}, &(10.8)\\
h^{(t)} &= \tanh\big(a^{(t)}\big), &(10.9)\\
o^{(t)} &= c + V h^{(t)}, &(10.10)\\
\hat{y}^{(t)} &= \text{softmax}\big( o^{(t)} \big), &(10.11)
\end{aligned}
$$

Where:

* $x$ are input sequences
* $o$ outputs of corresponding $x$
* $y$ training target for corresponding $x$
* $U$ weight matrix for input-to-hidden connections
* $W$ weight matrix for hidden-to-hidden connections
* $V$ weight matrix for hidden-to-output connections

First, **derivate of `softmax(x)`**, see this [post](http://peterroelants.github.io/posts/neural_network_implementation_intermezzo02/), using the quotient rule:

$$ \big(\frac{f}{g}\big)' = \frac{f'g - fg'}{g^2} $$

Let $f = e^x$, $g = \sum_i e^{x_i}$, $y = \text{softmax}(x)$ and $y_i$ be the result for $x_i$, $x \in R^n$.

For $i = j$:

$$
\begin{aligned}
\frac{\partial y_i}{\partial x_i} &= \frac{e^{x_i} g - e^{x_i} (0 + \cdots + e^{x_i} + 0 + \cdots + 0)}{g^2} \\
&= \frac{e^{x_i} g - e^{x_i} e^{x_i}}{g^2} \\
&= \frac{e^{x_i} (g - e^{x_i})}{g^2} \\
&= \frac{e^{x_i}}{g} \times \frac{g - e^{x_i}}{g}\\
&= y_i \times (1 - \frac{e^{x_i}}{g}) \\
&= y_i \times (1 - y_i)
\end{aligned}
$$

For $i \neq j$:

$$
\begin{aligned}
\frac{\partial y_i}{\partial x_j} &= \frac{0 \times g - e^{x_i} e^{x_j}}{g^2} \\
&= -\frac{e^{x_i}}{g} \frac{e^{x_j}}{g} \\
&= - y_i y_j
\end{aligned}
$$

Back to BPTT, we start from the node immediately preceding the final loss:

$$ \frac{\partial L}{\partial L^{t}} = 1 $$

Then we have

$$(\triangledown_{o^{t}}L)_i = \frac{\partial L}{\partial o^{t}_i} = \frac{\partial L}{\partial L^{t}} \frac{\partial L^{t}}{\partial o^{t}_i} = \hat{y}^{(t)}_i - 1_{1,y^{(t)}} $$